In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [10]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = np.array(json.loads(f_raw_input.read())["input_data"][0])
# data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.random.uniform(1,100, len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
# dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
# gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.tensor(np.median(dummy_data[dummy_data>30]))
# print(int(len(dummy_data)/2))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data[dummy_data>30])[int(len(dummy_data[dummy_data>30])/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data[dummy_data>30])[int(len(dummy_data[dummy_data>30])/2)])



In [11]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        # where part
        # to check: why cant do with num_fil_X with X>30 first?
        num_lowest = torch.sum((X<=30).double())
        num_fil_X = X.size()[1]-num_lowest
        min_X = torch.min(X)
        fil_X = torch.where(X>30, X, min_X-1)
        # fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
        
        count_less = torch.sum((fil_X < 0.99*self.w).double()) - num_lowest
        count_equal = torch.sum((torch.abs(fil_X-self.w)<=torch.abs(0.01*self.w)).double())
        half_len = torch.floor(torch.div(num_fil_X, 2))

        # not support modulo yet
        less_cons = count_less<half_len+2*(num_fil_X/2 - torch.floor(num_fil_X/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0 --> imply even length for sure
        lower_exist = torch.sum((torch.abs(fil_X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((fil_X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(fil_X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((fil_X<0.99*self.upper).double()) - num_lowest==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [12]:
# prover calculates settings, send to verifier
theory_output = torch.tensor(np.median(data[data>30]))
lower_to_median = torch.tensor(np.sort(data[data>30])[int(len(data[data>30])/2)-1])
upper_to_median = torch.tensor(np.sort(data[data>30])[int(len(data[data>30])/2)])

print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        # where part
        # to check: why cant do with num_fil_X with X>30 first?
        num_lowest = torch.sum((X<=30).double())
        num_fil_X = X.size()[1]-num_lowest
        min_X = torch.min(X)
        fil_X = torch.where(X>30, X, min_X-1)
        # fil_X = torch.tensor([ele[0] if ele[0]>30 else min_X -1  for ele in X[0]])
        
        count_less = torch.sum((fil_X < 0.99*self.w).double()) - num_lowest
        count_equal = torch.sum((torch.abs(fil_X-self.w)<=torch.abs(0.01*self.w)).double())
        half_len = torch.floor(torch.div(num_fil_X, 2))

        # not support modulo yet
        less_cons = count_less<half_len+2*(num_fil_X/2 - torch.floor(num_fil_X/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0 --> imply even length for sure
        lower_exist = torch.sum((torch.abs(fil_X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((fil_X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(fil_X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((fil_X<0.99*self.upper).double()) - num_lowest==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

Theory_output:  tensor(51., dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-582,2168],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":17417,"total_const_size":2720,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":2.0}},"ReLU",{"Floor":{"scale":4.0}},{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [13]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 2.083611011505127 seconds
Theory output:  tensor(51., dtype=torch.float64)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 51.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [16329468921151224777, 10175872942536559546, 2714029846925971291, 664661156343181352]]], 'proof': '1e964b382349da606ad48bbc473e4bfd0cdb0467511de4a114c3bfb14fd136c22808330170f7921a3f86587cffa5650530fe46c39ed13d4667bb1212bf7294832e25fab63a8da1314224b334055173e2ceb15dd59b49ea5cdfe8d4ca3d03bbcc210cde1f93ab50f94a17ceb47dc2b8b85f3a6afdbf79576eeebe7c144c3cfd210a36829bf027f1ebd798a779fc3631935d61ecc9e892f62694356d86653445a12ed30addea978f2f76f8fcf15a0b39de4c110865492ca4a9829741ef93e0869c1572c5b98debda8306208e3ac4f2791af9f907a1250053495f7d5ce89bcba2da04c2f3cbe5f9c0e66fd3a9424dc0ea8d2fd9442c26ce842ba376b80b92bfe9400c9770d8e37d0ac1af04fbc06c9f3b866d0dd5034cd1802a1c4247cd9a02a20b0fceac2ea6dcbd03d59bec590fbf573828b0dce1d315aa513a7758845305b64518de4eb175b6aac15dc8da520d13b901e7a1664a7aea3a98f405ae76eb512e9

In [14]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [16329468921151224777, 10175872942536559546, 2714029846925971291, 664661156343181352]]]
proof boolean:  1.0
proof result 1 : 51.0
verified
